In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
pas = pd.read_csv("../data/pas_boroughs.csv")
sas = pd.read_csv("../data/merged_filtered_sas.csv")
sas.rename(columns={"Borough": "borough"}, inplace=True)
sas = sas[sas['borough'] != 'Unknown']

In [70]:
sas

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Arrest Year,quarter,borough
0,Person search,2021-03-31 23:01:00+00:00,False,NaN,51.500886,-0.068284,Male,10-17,Other ethnic group - Not stated,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN,2021,1,Southwark
1,Person search,2021-03-31 23:06:00+00:00,False,NaN,51.537199,-0.076004,Male,10-17,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,NaN,NaN,2021,1,Hackney
2,Person search,2021-03-31 23:10:00+00:00,False,NaN,51.378427,-0.102657,Male,10-17,Other ethnic group - Not stated,White,Criminal Justice and Public Order Act 1994 (se...,Anything to threaten or harm anyone,A no further action disposal,NaN,NaN,2021,1,Croydon
3,Person search,2021-03-31 23:32:00+00:00,False,NaN,51.521962,-0.221082,Male,10-17,Black/African/Caribbean/Black British - Any ot...,Black,Police and Criminal Evidence Act 1984 (section 1),Evidence of offences under the Act,A no further action disposal,NaN,NaN,2021,1,Kensington and Chelsea
4,Person search,2021-04-01 00:30:00+00:00,False,NaN,51.621786,-0.292547,Male,10-17,Other ethnic group - Not stated,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,NaN,NaN,2021,2,Barnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55892,Person search,2023-09-30 22:00:00+00:00,False,NaN,51.558053,0.239651,Male,10-17,White - Any other White background,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,NaN,NaN,2023,3,Havering
55893,Person search,2023-09-30 22:15:00+00:00,False,NaN,51.562985,0.214369,Male,10-17,White - English/Welsh/Scottish/Northern Irish/...,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,NaN,NaN,2023,3,Havering
55894,Person search,2023-09-30 22:19:00+00:00,False,NaN,51.632352,-0.173649,Male,10-17,Other ethnic group - Not stated,Asian,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,NaN,NaN,2023,3,Barnet
55895,Person search,2023-09-30 22:56:00+00:00,False,NaN,51.514246,-0.131436,Male,10-17,Black/African/Caribbean/Black British - African,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,Arrest,NaN,NaN,2023,3,Islington


In [72]:
import pandas as pd

# Assume 'pas' and 'sas' are already loaded as pandas DataFrames

# Filter out rows where the borough is 'Unknown'
pas = pas[pas['borough'] != 'Unknown']
sas = sas[sas['borough'] != 'Unknown']

# Convert 'Date' to datetime in SAS
sas['Date'] = pd.to_datetime(sas['Date'])
sas['year_int'] = sas['Date'].dt.year
sas['quarter'] = sas['Date'].dt.quarter

# Shift the quarters for SAS to align with PAS definition
sas['year_int'] += (sas['quarter'] == 4).astype(int)
sas['quarter'] = sas['quarter'].map({1: 4, 2: 1, 3: 2, 4: 3})

sas_filtered = sas[['borough', 'year_int', 'quarter', 'Outcome', 'Object of search']]
sas_dummies = pd.get_dummies(sas_filtered, columns=['Outcome', 'Object of search'])

#Dummy variables
sas_grouped = sas_dummies.groupby(['borough', 'year_int', 'quarter']).sum().reset_index()

# Merging PAS Data
merged_data = pd.merge(pas, sas_grouped, on=['borough', 'year_int', 'quarter'], how='outer')

# Fill empty values with 0
merged_data.fillna(0, inplace=True)

# Removing rows with 'good_job' score equal to 0
merged_data = merged_data[merged_data['good_job'] != 0]

In [73]:
merged_data = merged_data[merged_data['good_job'] != 0]
merged_data

,borough,good_job,trust,fair_treatment,dealing_issues,quarter,year,year_int,Outcome_A no further action disposal,Outcome_Arrest,...,Outcome_Penalty Notice for Disorder,Outcome_Summons / charged by post,Object of search_Anything to threaten or harm anyone,Object of search_Articles for use in criminal damage,Object of search_Controlled drugs,Object of search_Evidence of offences under the Act,Object of search_Firearms,Object of search_Fireworks,Object of search_Offensive weapons,Object of search_Stolen goods
0,Barking and Dagenham,0.47,0.80,0.66,0.68,1.0,21-22,2021,120.0,17.0,...,0.0,5.0,1.0,0.0,77.0,13.0,0.0,4.0,37.0,13.0
1,Barking and Dagenham,0.44,0.77,0.66,0.64,2.0,21-22,2021,85.0,18.0,...,1.0,1.0,0.0,0.0,76.0,5.0,0.0,1.0,26.0,8.0
2,Barking and Dagenham,0.50,0.78,0.70,0.70,3.0,21-22,2021,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Barking and Dagenham,0.47,0.75,0.70,0.66,1.0,22-23,2022,85.0,26.0,...,0.0,4.0,0.0,0.0,56.0,9.0,0.0,0.0,46.0,20.0
4,Barking and Dagenham,0.50,0.77,0.71,0.66,2.0,22-23,2022,121.0,14.0,...,1.0,2.0,0.0,1.0,80.0,8.0,5.0,2.0,47.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Westminster,0.51,0.80,0.58,0.60,4.0,21-22,2022,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,Westminster,0.59,0.74,0.67,0.65,1.0,23-24,2023,52.0,11.0,...,0.0,0.0,0.0,4.0,23.0,0.0,0.0,0.0,18.0,23.0
353,Westminster,0.59,0.74,0.67,0.65,2.0,23-24,2023,40.0,15.0,...,1.0,0.0,0.0,0.0,11.0,2.0,0.0,0.0,20.0,27.0
354,Westminster,0.60,0.78,0.71,0.67,3.0,23-24,2023,25.0,17.0,...,0.0,0.0,0.0,1.0,13.0,0.0,0.0,1.0,7.0,23.0


In [74]:
merged_data.to_csv('merged_data.csv', index=False)